In [2]:
import pandas as pd
import gzip
import io
import os

In [7]:
df_bg = pd.read_parquet(r'../datasets/processed/google/business_google.parquet.gz')
df_by = pd.read_parquet(r'../datasets/processed/yelp/business_yelp.parquet.gz')
df_ug = pd.read_parquet(r'../datasets/processed/google/user_google.parquet.gz')
df_uy = pd.read_parquet(r'../datasets/processed/yelp/user_yelp.parquet.gz')
df_rg = pd.read_parquet(r'../datasets/processed/google/reviews_google.parquet.gz')
df_ry = pd.read_parquet(r'../datasets/processed/yelp/reviews_yelp.parquet.gz')

### KPI 1: Mejorar un 10% la accesibilidad de los restaurantes en Florida 

Criterio: Según el ratio de la relación de cantidad de restaurantes sobre cantidad de habitantes por estado, Florida es del menor ratio.

count business_id / habitantes 

In [8]:
df_bg

,gmap_id,name,latitude,longitude,avg_stars,state_id
0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,26.641377,-81.940545,5.00,Florida
1,0x88c2e4e34f1ed783:0x76c5da381c499d79,Fresh Point Country Buffet,27.867489,-82.702971,5.00,Florida
2,0x8890b9241e704667:0x3a1e565c17c00993,Hot Box,30.391411,-87.267220,4.20,Florida
3,0x88e635378f43352f:0xa1b53c63436fa428,Shell,29.183272,-81.889965,1.80,Florida
4,0x88d9ab98b5baad79:0x1a2e3f0642a55246,Baby Food Grill & Bar,26.010250,-80.148659,3.00,Florida
...,...,...,...,...,...,...
54901,0x89c3b2b8e9c8c357:0xb1b07826ef85a598,Park View Tavern,40.653743,-74.234148,4.26,New Jersey
54902,0x89c24d60acc3ffff:0xc8048546518281f6,The Brown Derby Cafe & Rental Hall,40.661250,-74.193971,4.73,New Jersey
54903,0x89c24d7a198a3541:0x3dcb8f429c053248,George's Lunch,40.654865,-74.177221,4.40,New Jersey
54904,0x89c3b290dcc12d6d:0x34ad59938efed577,Las Brasas Sports Bar and Restaurant,40.661939,-74.248363,3.81,New Jersey


In [9]:
# Diccionario con la población de California, Florida, Illinois y New Jersey desde 2015 hasta 2023
population_data = {
    'California': {
        2015: 39144818,
        2016: 39250017,
        2017: 39399349,
        2018: 39538223,
        2019: 39613506,
        2020: 39538223,
        2021: 39296476,
        2022: 39056079,
        2023: 38982847
    },
    'Florida': {
        2015: 20271272,
        2016: 20656589,
        2017: 20984400,
        2018: 21299325,
        2019: 21477737,
        2020: 21538187,
        2021: 21733312,
        2022: 22244823,
        2023: 22733312
    },
    'Illinois': {
        2015: 12859995,
        2016: 12802503,
        2017: 12778828,
        2018: 12741080,
        2019: 12671821,
        2020: 12812508,
        2021: 12671869,
        2022: 12518144,
        2023: 12470000
    },
    'New Jersey': {
        2015: 8958013,
        2016: 9005644,
        2017: 9032872,
        2018: 8908520,
        2019: 8882190,
        2020: 9288994,
        2021: 9261692,
        2022: 9288994,
        2023: 9290000
    }
}

# https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_population

In [18]:
# Calcular el número de negocios únicos por habitante por año
businesses_per_capita = {}
for state, population in population_data.items():
    # Filtrar los datos del DataFrame por estado
    df_filtered = df_bg[df_bg['state_id'] == state]
    # Calcular el número de negocios únicos
    unique_businesses = df_filtered['gmap_id'].nunique()
    # Calcular el número de negocios únicos cada 10000 habitantes en 2023
    businesses_per_capita[state] = unique_businesses / population[2023] * 10000

# Mostrar el diccionario resultante
businesses_per_capita

{'California': 6.668830524358572,
 'Florida': 5.845166775523074,
 'Illinois': 6.493985565356856,
 'New Jersey': 8.097954790096878}

### KPI 2: Evaluar calidad media de productos por categoría de restaurantes
Recomendación según top 20 categorias, según correlación entre categoria y stars a través del tiempo            
    Correlación entre las estrellas y las 20 categorías con mayor frecuencia        
    business_yelp.json.gz           

top 20 categorias ->  5 grupos de categorias
correlación entre grupos y stars
group by year

output: diccionario metrica por estado por año
kpi: aumento anual metrica 

KPIs:

Enfoque técnico: Optimización del alcance y calidad del servicio        
Mejorar un 10% la accesibilidad de los restaurantes en Florida      
    Cantidad de restaurantes por estado cada 1000 habitantes        
    business_google.json.gz         


Evaluar calidad media de productos por categoría de restaurantes
Recomendación según top 20 categorias, según correlación entre categoria y stars a través del tiempo            
    Correlación entre las estrellas y las 20 categorías con mayor frecuencia        
    business_yelp.json.gz           


Enfoque orgánico: Networking            
Mejorar la tasa de diferencia entre reviews y respuestas 
"Aumentar la calidad de la comunicación a través de un mejor ratio respuestas por comentarios y  disminuir el tiempo promedio de respuestas"

kpi = (ratio respuestas/review * rango tiempo) + sentiment_resp promedio

Datos disponibles: respuesta/review, tiempo promedio y sentiment_resp promedio
    Ratio: respuestas / reviews, respuestas tiempo promedio
    business_id, resp time != 0, sentiment_score !=0, date             
    reviews_google.parquet.gz           


Promover la tasa de retención de cliente            
    count distinct review por usuario 
    count distinct por rango por usuario        
    reviews_google.parquet.gz           
segun distribucion de datos, x > 1 es valido para decir que es cliente habitual?           
rangos?

Aumentar 10% reviews de usuarios poco influyentes y 1% muy influyente            
    Cantidad de usuarios inluyentes y poco influyentes         
    users_yelp.parquet.gz 
    

    
metricas        
ratio useful/review-count       
distribucion de reviews por mes

esperar dataset de dami     

Sistema de recomendación para empresas: Análisis de ubicación       
    Por cantidad competencia en la zona     
    Por calidad competencia en la zona      
    Por horario por atributo            
Sistema de recomendación para empresas: Publicidad      
    Promoción por gustos del usuario        
    Por gustos de usuarios similares        
    Por gustos de amigos del usuario        
